```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```

# Desafio: ¿Qué tan memorable es un video?

Como trabajo final para la materia, les proponemos resolver un desafio de memorabilidad de un video, el cual requerirá de utilizar todos los conceptos que revisamos en la materia **al mismo tiempo**: imagenes, audio y texto.

## ¿De que se trata el desafío?

Esta tarea se centra en el problema de predecir qué tan memorable es un video para un espectador. Requiere que los alumnos predigan automáticamente las puntuaciones de memorabilidad de los videos que reflejan la probabilidad de que se recuerde un video. Recibiran un extenso conjunto de datos de videos que van acompañados de anotaciones memorables, así como predictores (features) extraidos previamente para facilitarles el procesamiento. Las etiquetas (labels) se ha recopilado a través de pruebas de reconocimiento y, por lo tanto, es el resultado de una medición objetiva del rendimiento de la memoria.

<img src='https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Docs/memorability.PNG' width=600 />

*Creditos del desafio original:*

http://www.multimediaeval.org/mediaeval2019/memorability/

Mihai Gabriel Constantin, University Politehnica of Bucharest, Romania
Bogdan Ionescu, University Politehnica of Bucharest, Romania
Claire-Hélène Demarty, Technicolor, France
Quang-Khanh-Ngoc Duong, Technicolor, France
Xavier Alameda-Pineda, INRIA, France
Mats Sjöberg, CSC, Finland

## Direcciones

Deberán entrenar modelos computacionales capaces de inferir la memorabilidad de video a partir del contenido visual. Para hacerlo, podran optar por utilizar los videos directamente o utilizar caracteristicas (features) que fueron extraidas desde los mismos utilizando algunas de las técnicas que vimos en este curso. Utilizando estos datos, deberan evaluar la performance del modelo utilizando un set de datos de evaluación. Los modelos se evaluarán a través de métricas de evaluación estándar utilizadas en las tareas de clasificación y regressión (dependiendo del tipo de desafío que elijan).

## Anotaciones para el problema de memorabilidad

Cuentan con la anotaciones para el problema de memorabilidad. Dependiendo de su elección pueden optar por resolver un problema de clasificación o un problema de regresión. Esto lo veran refrejado en:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular. Valores altos son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con Score superior a 0.5 es marcado como memorable (1), sino es marcado como no memorable (0)

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/

In [ ]:
import pandas as pd

labels = pd.read_csv('Data/ground_truth.csv')

In [ ]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorabable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


## Elije tu propia aventura

En este desafío, les propongo elegir la dificultad con la que quieren trabajar para solucionar el problema en cuestión. Dependiendo de la elección, son las herramientas que tendrán a su disosición para resolver el problema. Pueden utilizar algunas de las sugerencias, todas las sugerencias, o probar alguna técnica que conozcan y que les parezca interesante de probar.

### Resolver el desafío utilizando los datos de origen

Esta configuración del desafío es la más similar a la vida real, y solo contaran con un conjunto de videos en formato MP4 como datos de entrada junto con sus anotaciones de memorabilidad en formato CSV. Cualquier procesamiento de datos deberá realizarse utilizando esta información únicamente. Dado que algunos procesamientos de datos sobre videos pueden ser complejos, adicionalmente contarán además con:
 - Imágenes extraidas cada 2 segundos por cada secuencia a analizar
 - Tracks de audio disponibles en formato WAV listos para utilizar

A pesar de que el video a sido separado en imagenes y audio, resolver el desafío en utilizando estos datos será una tarea compleja. Sin embargo, en el contexto de sus organizaciones, normalmente se encontrarán con esta dificultad.

#### Videos

En el directorio */Data/Raw/sources* encontraran todas las secuencias utilizadas en este desafío. El archivo *'Data/ground_truth.csv'* contine los nombres de las peliculas a las que corresponde cada fragmento en caso que quieran revisarlos.

#### Audios

En el directorio */Data/Raw/audios* encontraran todos los audios correspondientes a cada una de las secuencias de los videos. Los audios están en formato wav, que si bien ocupan mayor espacio, son más sencillos de procesar. 

Algunas ideas:
 - Aplicar técnicas de speech-to-text.
 - Aplicar técnicas de embeddings de audio.
 - Clasificar primero el audio (en por ejemplo 'instrumento musical', 'personas charlando', 'exterior', 'aire libre') y luego utilizarlos para resolver el problema.

#### Frames

En el directorio */Data/Raw/frames* encontraran todos los frames extraidos de cada una de las secuencias. Los frames están disponibles cada 2 segundos por lo cual disponen de 5 frames por cada secuencia.

Algunas ideas:
 - Aplicar modelos de convolución directamente sobre las imagenes.
 - Aplicar técnicas de descripción de imagenes utilizando Servicios Cognitivos
 - Reconocimiento de lugares claves.
 - Los fragmentos tienen personas famosas, ya que son actores de cine y televisión. ¿Será importante?
 - Colores de la escena.

### Resolver el desafío utilizando predictores preprocesados

En esta configuración del desafio, ya hay bastante trabajo realizado. Los videos han sido preprocesados para generar un conjunto de predictores que pueden ayudar a resolver el problema. Estos predictores se entregan como un conjunto de archivos en formato CSV. 

A pesar de que esta opción pueda parecer sencilla, ofrece sus propios desafíos ya que tienen a disposición muchos predictores pero en dominios distintos: audio, texto, embeddings, etc. Deberán desarrollar una técnica que pueda hacer sentido de esto en su conjunto.

A continuación detallamos los predictores con los que contarán:

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/audio_vectors.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/c3d_vectors.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/colors_features.csv --directory-prefix ./Data/Features/

#### Descripciónes de las imagenes

Cuentan con las descripciones de cuadros (frames) de cada una de las secuencias. Estos frames están extraidos cada 1 segundo y las descripciones se generaron utilizando la API de Computer Vision de Azure Cognitive Services. Cada una de las descripciones forma una oración, y por lo tanto, por cada secuencia tendrán disponible un parrafo con 10 oraciones describiendo cada una de las escenas:

In [ ]:
import pandas as pd
captions = pd.read_csv('Data/Features/caption_features.csv')

In [ ]:
captions.head(2)

,sequence_name,cc
0,127_hours_2000_2010_1,a man in a suit and tie standing in front of a...
1,127_hours_2182_2192_5,a man holding a toothbrush in his mouth. a wom...


#### Colores de las imágenes

Los colores en una imágen pueden ser representativos de lo que uno está viendo. Por ejemplo, en una secuencia de terror, las imagenes con fondos oscuros quizás prevalezcan que con colores claros. Para capturar esto, cuentan con anotaciones para cada secuencia, para cada cuadro (frame de la secuencia) con los valores correspondientes al color más dominante en el fondo y el color más dominante en primer plano. Noten que en este caso, disponen de esta información por cada uno de los cuadros (es decir, 5 anotaciones por cada secuencia y por lo tanto deberán elejir una estrategia para quedarse con 1 solo valor por secuencia. Por ejemplo: el color más frecuente, o el color más raro. También pueden optar por utilizar variables categoricas en el sentido "contine_fondo_oscuro" significando una secuencia en que el fondo en algún cuadro es oscuro/negro.

In [4]:
import pandas as pd
colors = pd.read_csv('Data/Features/colors_features.csv')

In [5]:
colors.head(5)

,sequence_name,dominant_background,dominant_foreground
0,127_hours_2000_2010_1,Black,Black
1,127_hours_2000_2010_1,Grey,Black
2,127_hours_2000_2010_1,Black,Black
3,127_hours_2000_2010_1,Black,Black
4,127_hours_2000_2010_1,Black,Black


#### Vectores de audio

Se corresponden a las representaciones (embeddings) utilizados por la red YAMNet que vimos en el curso. YAMNet es un modelo que utiliza la misma ténica que las CNN pero al construir imágenes a partir de audio (llamados parches o patchs). Cada uno de estos patchs es el resultado de calcular espectrogramas de audio, creando así parches de imágenes 2D para utilizar en nuestro modelo.

Los vectores generados por YAMNet son vectores en un espacio de 1024-D


In [ ]:
import pandas as pd
audio_embeddings = pd.read_csv('Data/Features/audio_vectors.csv')

In [ ]:
audio_embeddings.head(2)

,Unnamed: 0,sequence_name,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,0,127_hours_2000_2010_1,0.0,0.796399,0.009879,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.492673,0.0,0.0,0.0,0.391332,0.0,0.0
1,1,127_hours_2000_2010_1,0.0,0.000000,0.000000,0.003163,0.010787,0.0,0.0,0.0,...,0.0,0.0,0.0,0.173500,0.0,0.0,0.0,0.177809,0.0,0.0


#### Vectores de la red C3D (convoluciones en 3D)

Se corresponden a las representaciones aprendidas al aplicar un modelo de convolución 3D sobre el input del video. Estas convoluciones no solo tienen en cuenta localidad a nivel de pixeles en las imagenes sino que también localidad a nivel espacio-temporal en el transcurso del video.

<img src='https://github.com/santiagxf/M72109/blob/master/Desafio/Docs/3D-timedistributed.PNG?raw=1' width=500 />

Referencias sobre este modelo: https://arxiv.org/pdf/1711.11248.pdf<br />
Implementación por el equipo de Facebook AI: https://github.com/facebookresearch/VMZ

En en caso de las representaciones provistas aquí, corresponden al caso B) que se muestra en la imagen superior, donde se aplicaron convoluciones 3D (alto, ancho y tiempo) en las primeras capas para capturar relaciones temporales a alto nivel y luego se continuó con convoluciones 2D de forma similar a lo que vimos en este curso.

Los embeddings generados son vectores en un espacio dimensional 4096-D

In [12]:
import pandas as pd
video_embeddings = pd.read_csv('Data/Features/c3d_vectors.csv')

In [13]:
video_embeddings.head(2)

,Unnamed: 0,sequence_name,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0,127_hours_2000_2010_1,0.82588,6.3702,0.0,0.34248,0.75896,1.7976,0.1524,0.0,...,3.9249,0.0,5.6224,1.7332,2.4912,3.27780,1.7001,0.0000,1.9559,6.1765
1,1,127_hours_2182_2192_5,0.00000,2.2294,0.0,0.00000,1.53370,1.2082,2.6512,0.0,...,0.3997,0.0,3.8342,1.7536,1.8910,0.91983,0.0000,2.0523,1.0713,1.4447
